# Day 3 - Practical ✍
## Advanced ML Workshop
---
## Titanic Dataset
Welcome to the third practical session of SPAI's Advanced Machine Learning Workshop. In this practical, you will experience model selection, hyperparameter tuning techiniques as well as some useful feature selection techniques. Finally, you will learn how to save your model in a `.pkl` file so that you can work on it later. The goal of this dataset is to predict the survivability of passenger in the Titanic ship.

## ⚙ *Basic Initialization*
Please do not make any changes to this section.   
This section is required for the demo to work and not running it would result in errors 🚨.   
However, if you would like to understand the code, feel free to do so 😊

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore') # Filter out warnings

In [2]:
titanic = pd.read_csv('https://raw.githubusercontent.com/SPAI-Team/Advanced-ML-Workshop-2021/main/Datasets/titanic_cleaned.csv', index_col=0)

In [3]:
X, y = titanic.drop(columns=['survived']), titanic['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

---

## Exercise 1️⃣
In this exercise, we will be attempting to select the best model for this dataset.

> Tip 💡: Try looping through all the models! Feel free to refer to the tutorials.

#### Task
1. Define a `models` dictionary that stores the respective name of the model as the key, the instance of the actual model as the value
    - Example, `Logistic Regression` key should be filled with `LogisticRegression()` 
    - Feel free to refer to Demonstration 1️⃣

In [5]:
# Test the following models. Evaluate which models perform the best 💯!
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# ensemble models are usually tested too. However, it would take far too long!
# from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor

### Write your code here ###
# Declare a Dictionary of models with their name
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'K Neighbours': KNeighborsClassifier(),
    'SVC': SVC()
}
### End ###

#### Task
1. Import `cross_validate` from `sklearn.model_selection`
2. Inside the for loop, apply cross validation with the `cross_validate` function you just imported and save the result in the variable `results`. In the `cross_validate` function, parse the following:
    - pass the model, `models[model]`
    - `X_train` and `y_train`
    - `cv=5`, `scoring=scoring_metrics`, `return_train_score=True`
    - Set `n_jobs` to `-1` for faster training speeds
3. Store all the necessary result into their necessary keys in the `scores` dictionary
    - The pattern is as follows, `scores[`*key*`].append(np.mean(results[`*key with underscore*`]))`
    - Feel free to refer to Demonstration 1️⃣

In [8]:
results

{'fit_time': array([0.02140141, 0.03160548, 0.01785731, 0.01607227, 0.01774311]),
 'score_time': array([0.00440311, 0.00373721, 0.00364184, 0.00368953, 0.00210428]),
 'test_accuracy': array([0.76811594, 0.83211679, 0.7810219 , 0.77372263, 0.75182482]),
 'test_f1': array([0.6       , 0.72289157, 0.58333333, 0.62650602, 0.51428571])}

In [9]:
scoring_metrics = ['accuracy', 'f1']

scores = {
    'model': [],
    'train accuracy': [],
    'test accuracy': [],
    'train f1': [],
    'test f1': [],
    'fit_time': [],
}


### Write your code here ###
# import the cross_validate library from model_selection in sklearn library
from sklearn.model_selection import cross_validate

# For loop through the models
for model in models.keys():
    # Initialize your results
    results = cross_validate(models[model], X_train, y_train, cv=5, scoring=scoring_metrics, return_train_score=True, n_jobs=-1)



    # Append the corresponding results into scores
    scores['model'].append(model)
    scores['train accuracy'].append(np.mean(results['train_accuracy'])) # For train accuracy
    scores['test accuracy'].append(np.mean(results['test_accuracy'])) # For test accuracy
    scores['train f1'].append(np.mean(results['train_f1'])) # For train f1
    scores['test f1'].append(np.mean(results['test_f1'])) # For test f1
    scores['fit_time'].append(np.mean(results['fit_time'])) # For fit time

### End ###


# Allows us to view scores in a nice format
pd.DataFrame(scores)

,model,train accuracy,test accuracy,train f1,test f1,fit_time
0,Logistic Regression,0.790090,0.781360,0.625176,0.609403,0.022096
1,Decision Tree,0.954445,0.731757,0.921864,0.532172,0.004927
2,K Neighbours,0.825437,0.768201,0.686208,0.575232,0.005847
3,SVC,0.788995,0.788628,0.623589,0.620149,0.016756


---

## Exercise 2️⃣
In this exercise, you will learn how to use both `GridSearchCV` and `RandomSearchCV`

In [10]:
dt = DecisionTreeClassifier()

### Grid Search Cross Validation
#### Task
1. Import `GridSearchCV` from `sklearn` under `model_selection`
2. Get the params from the Decision Tree Classifier
    - Do note that the `DecisionTreeClassified()` is already defined with the `dt` variable

In [11]:
### Write your code here ###
# Import GridSearchCV from sklearn under model_selection
from sklearn.model_selection import GridSearchCV

# Get the parameters you can configure
dt.get_params()

### End ###

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

#### Task
1. Set each key the array of values to experiment with. Set the following keys with the following array of values:
    - `min_samples_leaf` with values ranging from 2 to 6
    - `max_depth` with values ranging from 2 to 6
    - `min_samples_split` with values ranging from 1 to 5
    - `max_features` with values ranging from 5 to 8
2. Peform grid search with the `params` dictionary with the decision tree model. 
    - Set `n_jobs` to `-1` for faster training speeds
    - After which, fit it with `X_train` and `y_train`
    - Save the grid search instance to `result` variable
3. Print the best score and best parameters

In [12]:
'''
Hint:
Range creates an iterable from the start point of a until b-1
range(a,b) = [a, a+1, a+2, ..., b-2, b-1]
'''
### Write your code here ###
# Parameters to test 
params = {
    'min_samples_leaf': range(2,6),
    'max_depth': range(2, 6),
    'min_samples_split': range(1,5),
    'max_features': range(5, 8)
}

# Runs Grid Search
result = GridSearchCV(dt, params, cv=5, n_jobs=-1).fit(X_train, y_train)

# Returns Scores and parameters respectively
print('Best Score is:', result.best_score_)
print('Best Parameters:\n', result.best_params_)

### End ###

Best Score is: 0.7974505448005924
Best Parameters:
 {'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}


### Random Search Cross Validation

#### Task
1. Import `RandomSearchCV` from `sklearn` under `model_selection`

In [13]:
### Write your code here ###
# Import GridSearchCV from sklearn under model_selection
from sklearn.model_selection import RandomizedSearchCV

### End ###

#### Task
1. Peform grid search with the `params` dictionary with the decision tree model. 
    - Set `n_jobs` to `-1` for faster training speeds
    - After which, fit it with `X_train` and `y_train`
    - Save the grid search instance to `result` variable

In [14]:
### Write your code here ###
# Runs Grid Search
result = RandomizedSearchCV(dt, params, cv=5, n_jobs=-1).fit(X_train, y_train)

### End ###

# Returns Scores and parameters respectively
print('Best Score is:', result.best_score_)
print('Best Parameters:\n', result.best_params_)

Best Score is: 0.7814027292922882
Best Parameters:
 {'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 6, 'max_depth': 5}


---

## Exercise 3️⃣
In this exercise, you will learn how to carry out feature selection.

In [15]:
dt = DecisionTreeClassifier()

### Feature Importance

#### Task
1. Train a Decision Tree Classifier model with `X_train` and `y_train`
2. Display Feature Importance
    - Feel Free to Refer to Demonstration 3️⃣

In [16]:
### Write your code here ###
# Train Decision Tree Model
model = dt.fit(X_train, y_train)

# Display Feature Importance
pd.DataFrame({
    'columns': X_train.columns,
    'feature importance': model.feature_importances_ # Call feature importance method here
}).sort_values('feature importance', ascending=False)

### End ###

,columns,feature importance
1,age,0.306653
4,fare,0.289731
5,sex_male,0.269497
0,pclass,0.067540
2,sibsp,0.032437
7,embarked_S,0.026557
6,embarked_Q,0.007585
3,parch,0.000000


### Coeffecient

#### Task
1. Train a Logistic Regression model with `X_train` and `y_train`
2. Display the coeffecient
    - Hint: coeffecient is a 2D array

In [17]:
### Write your code here ###
# Train Logistic Regression Model
model = LogisticRegression().fit(X_train, y_train)

# Display Coeffecient
pd.DataFrame({
    'columns': X_train.columns,
    'coef': model.coef_[0] # Call coef method here
}).sort_values('coef', ascending=False)

### End ###

,columns,coef
4,fare,1.266971
3,parch,0.000281
1,age,-0.131962
2,sibsp,-0.384548
0,pclass,-0.432765
7,embarked_S,-0.471603
6,embarked_Q,-0.574393
5,sex_male,-2.379805


### RFECV

#### Task
1. Import `RFECV` from scikit-learn in `feature_selection`
2. Initialize RFECV to result and do the following:
    - Parse the Decision Tree Classifier Model which has been initialized with variable `dt`
    - Set `scoring='accuracy'` and set `n_jobs=-1` for faster fit time
    - Apply `.fit()` method parsing `X_train` and `y_train`
3. Get the ranking of each variable
    - Feel Free to Refer to Demonstration 4️⃣

In [18]:
### Write your code here ###
# Import RFECV from sklearn under feature_selection
from sklearn.feature_selection import RFECV

# Performs RFECV
result = RFECV(dt, scoring='accuracy', n_jobs=-1).fit(X_train, y_train)

# Display Ranking of features
pd.DataFrame({
    'columns': X_train.columns,
    'ranking': result.ranking_ # Call ranking method here
}).sort_values('ranking')

### End ###

,columns,ranking
0,pclass,1
1,age,1
2,sibsp,1
3,parch,1
4,fare,1
5,sex_male,1
6,embarked_Q,1
7,embarked_S,1


---

## Exercise 4️⃣
In this exercise, we will be saving and loading models.

#### Task
1. Import joblib
2. Train a Decision Tree Classifier model
3. Save the model using `joblib`
    - Use the `dump` method in the joblib library
    - Parse the trained model and a file name of your choosing
    - Make sure that the file name ends with `.pkl`

In [19]:
### Write your code here ###
# Import Joblib library
import joblib

# Train Decision Tree Classifier
model = DecisionTreeClassifier().fit(X_train, y_train)

# Save the model
joblib.dump(model, 'day 3.pkl')

### End ###

['day 3.pkl']

#### Task
1. Load the model from the `.pkl` file with the file name used in the previous cell.
    - Be sure to load the model into the variable `model`
2. Generate prediction from the model
    - run `.predict` method using `X_test`
3. Use your testing data to test the accuracy of the model

In [20]:
### Write your code here ###
# Load the model from the saved .pkl file
model = joblib.load('day 3.pkl')

# Generate predictions
y_pred = model.predict(X_test)

# Test accuracy of the model
accuracy_score(y_test, y_pred)

### End ###

0.7848837209302325

# Conclusion
Congratulations! You have completed Practical Notebook for Day 3 of SPAI Advanced Machine Learning Workshop.  If you have any doubts or require any clarification feel free to approach us through our [Instagram](https://www.instagram.com/spai.sp/) or [Discord Server](https://discord.gg/zPYJMGfQFa)*(remember to verified yourself)*.

---
> ### Feel Free to Join the Subsequent **SPAI Machine Learning Competition** to earn **valuable Prize💰 and Experience🏆** if you have not done so!🥳🥳
[ML Comp Sign Up Link](https://docs.google.com/forms/d/e/1FAIpQLSchLDXDAY0LqM6fuRDyQwdRNbVT4FYrgDtqthEIfYpFvpWMAg/viewform) *(If it has not been closed)*